In [1]:
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta, date
from humanize import naturalsize
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Qiita's resource allocation - quick update from previous version

After the 2023.10 release we noticed that:
1. `job-output-folder` `VALIDATE` command didn't have valid request because those jobs do not have sample/column values
2. The default during a resource allocation for time is minutes and the calculations were done in seconds

# Loading data

First you will need to run `generate-allocation-summary.py` in Qiita as the qiita user (or whatever user runs qiita in your system). The resulting file would be: `job_[date].tsv.gz`.

The generated file will have these columns: `['JobID', 'ElapsedRaw', 'MaxRSS', 'Submit', 'Start', 'MaxRSS.1', 'CPUTimeRAW', 'ReqMem', 'AllocCPUS', 'AveVMSize', 'QiitaID', 'external_id', 'sId', 'sName', 'sVersion', 'cId', 'cName', 'samples', 'columns', 'input_size', 'extra_info'],`.

In [2]:
m1g = 2**30
df = pd.read_csv('jobs_2023-10-04.tsv.gz', sep='\t', dtype={'extra_info': str})
df['ElapsedRawTime'] = pd.to_timedelta(df.ElapsedRawTime)

In [3]:
# for reference for the next iteration of this notebook
f'There are {df.shape[0]} successful jobs since we moved to barnacle2 and the largest external_id is: {df.external_id.max()}'

'There are 101147 successful jobs since we moved to barnacle2 and the largest external_id is: 1581986'

# 1. Getting the default values for `job-output-folder` `VALIDATE`

In [4]:
cname = 'Validate'
_df = df[(df.cName == cname)].copy()

summary = _df[_df['samples'].isnull() & _df['columns'].isnull()].groupby(
    ['cName', 'sName'])[['ElapsedRawTime', 'MaxRSSRaw']].agg(['count', 'min', 'max']).copy()

summary[('MaxRSSRaw', 'min')] = summary[('MaxRSSRaw', 'min')].apply(naturalsize)
summary[('MaxRSSRaw', 'max')] = summary[('MaxRSSRaw', 'max')].apply(naturalsize)

display(summary)

# New allocation: -p qiita -N 1 -n 1 --mem 100mb --time 00:40:00

ElapsedRawTime  \
                                                            count   
cName    sName                                                      
Validate BIOM type - BIOM                                     687   
         Diversity types - FeatureData                          6   
         Diversity types - alpha_vector                       123   
         Diversity types - distance_matrix                    117   
         Diversity types - ordination_results                 107   
         Sequencing Data Type - Demultiplexed                  43   
         Sequencing Data Type - FASTA                           2   
         Sequencing Data Type - FASTQ                          32   
         Sequencing Data Type - SFF                             1   
         Sequencing Data Type - per_sample_FASTQ               73   
         Visualization types - q2_visualization               133   
         qtp-job-output-folder - job-output-folder            228   

                                                                    \
                                                               min   
cName    sName                                                       
Validate BIOM type - BIOM                          0 days 00:00:55   
         Diversity types - FeatureData             0 days 00:01:20   
         Diversity types - alpha_vector            0 days 00:01:12   
         Diversity types - distance_matrix         0 days 00:00:37   
         Diversity types - ordination_results      0 days 00:00:39   
         Sequencing Data Type - Demultiplexed      0 days 00:00:35   
         Sequencing Data Type - FASTA              0 days 00:00:56   
         Sequencing Data Type - FASTQ              0 days 00:00:41   
         Sequencing Data Type - SFF                0 days 00:01:09   
         Sequencing Data Type - per_sample_FASTQ   0 days 00:00:36   
         Visualization types - q2_visualization    0 days 00:00:36   
         qtp-job-output-folder - job-output-folder 0 days 00:00:31   

                                                                   MaxRSSRaw  \
                                                               max     count   
cName    sName                                                                 
Validate BIOM type - BIOM                          0 days 01:03:49       687   
         Diversity types - FeatureData             0 days 00:02:49         6   
         Diversity types - alpha_vector            3 days 04:36:54       123   
         Diversity types - distance_matrix         0 days 00:03:55       117   
         Diversity types - ordination_results      0 days 00:03:19       107   
         Sequencing Data Type - Demultiplexed      0 days 00:12:23        43   
         Sequencing Data Type - FASTA              0 days 00:02:23         2   
         Sequencing Data Type - FASTQ              0 days 01:50:44        32   
         Sequencing Data Type - SFF                0 days 00:01:09         1   
         Sequencing Data Type - per_sample_FASTQ   0 days 18:13:21        73   
         Visualization types - q2_visualization    0 days 00:24:56       133   
         qtp-job-output-folder - job-output-folder 0 days 00:04:06       228   

                                                                        
                                                         min       max  
cName    sName                                                          
Validate BIOM type - BIOM                           222.8 MB   82.0 GB  
         Diversity types - FeatureData              331.4 MB  384.3 MB  
         Diversity types - alpha_vector             289.3 MB  101.5 GB  
         Diversity types - distance_matrix          122.7 MB   12.5 GB  
         Diversity types - ordination_results       117.2 MB    2.9 GB  
         Sequencing Data Type - Demultiplexed        83.4 MB  517.4 MB  
         Sequencing Data Type - FASTA                79.8 MB   83.6 MB  
         Sequencing Data Type - F

# 2. Updates for the seconds to minute confusion


=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem (2**30)+({samples}*150000) '
            || '--time 240'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem (2**30)+({samples}*150000) '
           || '--time 4'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'delete_sample_or_column';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem {samples}*10000000'
            || '--time 61200'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem {samples}*10000000'
           || '--time 1020'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Sequence Processing Pipeline';

=============
=============
     allocation = '-p qiita -N 1 -n 1 --mem 4g --time 900'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 --mem 4g --time 15'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Filter samples from table [filter_samples]';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem (2**31)+({input_size}*6) if
(2**31)+({input_size}*6) < 13958643712 else 13958643712 '
            || '--time 2400'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem (2**31)+({input_size}*6) if
(2**31)+({input_size}*6) < 13958643712 else 13958643712 '
           || '--time 40'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Rarefy table [rarefy]';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem 14g'
            || '--time 360'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem 14g '
           || '--time 6'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Alpha diversity (phylogenetic) [alpha_phylogenetic]';


=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem (2**33)+(2**30)+(({samples}*{columns}*{input_size})/4500000)'
            || '--time 1800'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem
(2**33)+(2**30)+(({samples}*{columns}*{input_size})/4500000) '
           || '--time 30'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Visualize and Interact with Principal Coordinates Analysis
Plots [plot]';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem (2**32)+(({samples}*{columns}*{input_size}')/20000)'
            || '--time 90000'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem (2**32)+(({samples}*{columns}*{input_size})/20000) '
           || '--time 1500'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Alpha rarefaction curves [alpha_rarefaction]';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem 2*(2**30)+{input_size} if 2*(2**30)+{input_size} < 16*(2**30) else 16*(2**30)'
            || '--time 36000'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem 2*(2**30)+{input_size} if 2*(2**30)+{input_size}
< 16*(2**30) else 16*(2**30) '
           || '--time 600'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'Trimming';

=============
=============
     allocation = '-p qiita -N 1 -n 1 '
            || '--mem (2**30)+({samples}*{columns}*2000)'
            || '--time 2300'

UPDATE qiita.processing_job_resource_allocation set
    allocation = '-p qiita -N 1 -n 1 '
           || '--mem (2**30)+({samples}*{columns}*2000) '
           || '--time 39'
    WHERE job_type = 'RESOURCE_PARAMS_COMMAND' and
    name = 'update_sample_template';